In [ ]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_models_and_visualize_SMOTE(df, target_variable_name, model_or_models, columns_to_drop=None):
    if columns_to_drop is not None:
        df = df.drop(columns=columns_to_drop)
    
    x = df.drop(target_variable_name, axis=1)
    y = df[target_variable_name]
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    
    if not isinstance(model_or_models, dict):
        # If a single model is passed, convert it to a dictionary with a default name
        model_or_models = {"Model": model_or_models}
    
    scores = {}
    
    for name, model in model_or_models.items():
        # If the model requires SMOTE or Borderline SMOTE, incorporate it into a pipeline
        if 'SMOTE' in name or 'Borderline SMOTE' in name:
            if 'SMOTE' in name:
                resampling = SMOTE()
            else:
                resampling = BorderlineSMOTE(kind='borderline-1')
            pipeline = Pipeline([('Resampling', resampling), (name, model)])
            model_to_fit = pipeline
        else:
            model_to_fit = model
        
        model_to_fit.fit(X_train, y_train)
        predicted = model_to_fit.predict(X_test)
        report = classification_report(y_test, predicted, output_dict=True)
        scores[name] = report['macro avg']['f1-score']
        
        print(f"{name} Classification Report:\n", classification_report(y_test, predicted))
    
    # Visualization
    scores_df = pd.DataFrame(scores.items(), columns=['Model', 'F1 Score'])
    plt.figure(figsize=(10, 6))
    sns.barplot(x='F1 Score', y='Model', data=scores_df, palette='viridis')
    plt.title('F1 Scores by Model')
    plt.xlabel('F1 Score')
    plt.ylabel('Model')
    plt.xlim(0, 1)
    plt.show()
    
    return scores

#
# models = {
#     "Logistic Regression": LogisticRegression(max_iter=1000),
#     "Random Forest": RandomForestClassifier(n_estimators=100)
# }
# evaluate_models_and_visualize(df, 'Target', models, ['Column1', 'Column2'])
